## Part 0


### Setup a Docker image: install a Data Manipulation framework (PySpark or Pandas or any other that will help answer the challenge) and anything else you need. Include the Dockerfile in the submission.

In [142]:
import csv 
import json
import os 
import pandas as pd 
import sys 
from itertools import chain

## Part I 

#### Mapping of credit_card_vendor to list of card prefix

In [143]:
# prefix_vendor = list of credit first digits that are representing this vendor.
maestro = ['5018', '5020', '5038', '56']
mastercard = ['51', '52', '54', '55', '222']
visa = ['4']
amex = ['34', '37']
discover = ['6011', '65']
diners = ['300', '301', '304', '305', '36', '38']
jcb16 = ['35']
jcb15 = ['2131', '1800']

In [144]:
# Make dictionary of vendors 
vendors = {'maestro' : maestro,
           'mastercard' : mastercard,
           'visa' : visa,
           'amex' : amex,
           'discover' : discover,
           'diners' : diners,
           'jcb16' : jcb16,
           'jcb15' : jcb15}

In [145]:
# vendors

In [146]:
# Map all credit first digits to the vendor to a flattened list 
credit_cards = maestro + mastercard + visa + amex + discover + diners + jcb16 + jcb15

In [176]:
credit_cards

['5018',
 '5020',
 '5038',
 '56',
 '51',
 '52',
 '54',
 '55',
 '222',
 '4',
 '34',
 '37',
 '6011',
 '65',
 '300',
 '301',
 '304',
 '305',
 '36',
 '38',
 '35',
 '2131',
 '1800']

### Import, load, and return the fraud file

In [148]:
fraud_file = ('datasets/fraud')

In [149]:
def get_fraud_file(file):
    '''
    NEED TO ADD DOCSTRING
    TAKES IN A FILE, PARSES OUT RELEVANT CC INFO
    RETURNS LIST OF CREDIT CARD NUMBERS
    '''
    with open(file) as f:
        # Remember, not every row has a 'state' value
        # Only a few rows do
        # From this file we mainly care about the credit card number
        fraud_data = csv.reader(f)
        for index, row in enumerate(fraud_data):
            if len(row) > 2:
                state = row[2]# row[2] State (not always there)
            else:
                # Insert NULL for state
                state = None
            fraud_dataset = [x[0] for x in fraud_data]
    
    return fraud_dataset


In [150]:
fraud_dataset = get_fraud_file(fraud_file)
fraud_dataset

['4013901026491',
 '345936465222676',
 '4829769584081989',
 '30340825930914',
 '4302964471893676356',
 '4897487775841494',
 '3504134000927360',
 '343568314271448',
 '36795457706136',
 '213105387849185',
 '4158823194013915',
 '3557121809488039',
 '4411710233936193',
 '3593952520772269',
 '562632689286',
 '675979363636',
 '3506507615619043',
 '3560888773289883',
 '5128258542512744',
 '4673436419900968',
 '6011531195808785',
 '3518829214836604',
 '676342610232',
 '503886101024',
 '346979098869785',
 '213175362057328',
 '36828949701611',
 '6011712584028793',
 '4408482692580798',
 '4716466790794127',
 '4581622575175591',
 '4534971465733250',
 '348623602717035',
 '3562843917409457',
 '3503861795779124',
 '3548979454205467',
 '4562223449118630453',
 '5160813237111985',
 '3500558499395149',
 '4130253854633',
 '3560320084927241',
 '213172153235353',
 '343924572576715',
 '4183170112593',
 '213151645209580',
 '5579434556811330',
 '4912298956727635308',
 '5178628606737917',
 '4614910245590',
 '356

In [151]:
len(fraud_dataset)

1011

# Part I

### Sanitize data of both transaction-001.zip and transaction-002.zip by removing transactions where column `credit_card_number` is not part of the previous provided list. For example, a credit card that starts with `98` is not a valid card and should be discarded from the sanitized dataset.


In [152]:
file_list=['datasets/transaction-001','datasets/transaction-002']

In [153]:
def load_transaction_files(file_list):
    main_dataframe = pd.DataFrame(pd.read_csv(file_list[0]))
    for i in range(1,len(file_list)):
        data = pd.read_csv(file_list[i])
        df = pd.DataFrame(data)
        main_dataframe = pd.concat([main_dataframe,df],axis=0)
    transaction_files = main_dataframe.reset_index(drop=True)
    return transaction_files

In [154]:
transaction_files = load_transaction_files(file_list)
transaction_files

,credit_card_number,ipv4,state
0,30213196611688,192.168.216.212,AZ
1,4938684086769,172.29.189.27,CA
2,213128820373753,172.22.174.184,AR
3,3554681727155351,172.24.27.121,ID
4,4339158798023,10.87.17.223,SC
...,...,...,...
499995,4407827161521023,192.168.92.89,NM
499996,4797462133093618,10.228.168.20,WI
499997,3595014020561169,172.21.190.37,RI
499998,30416095371458,192.168.68.115,MO


#### Sanity check that the files loaded properly and the dataframe is of the correct size 
Also reset the index to show it as one complete file instead of two files each of size 250000 entries. 


In [155]:
len(transaction_files) # Should be 500,000

500000

In [156]:
# transaction_files.drop_duplicates()

In [157]:
def get_prefix_column(transaction_files):
    numbers = transaction_files["credit_card_number"]
    start_nums = []
    for index, number in enumerate(numbers):
        str_num = str(number)
        strt_val = list(filter(str_num.startswith, credit_cards)) 
        start_nums.append(strt_val)
    start_vals = list(chain(*start_nums)) 
    start_vals = [int(i) for i in start_vals]
    return start_vals

start_vals = get_prefix_column(transaction_files)
start_vals

[4,
 2131,
 35,
 4,
 34,
 1800,
 305,
 4,
 2131,
 36,
 4,
 35,
 4,
 4,
 2131,
 35,
 1800,
 51,
 35,
 35,
 4,
 35,
 301,
 6011,
 34,
 4,
 36,
 35,
 4,
 6011,
 4,
 2131,
 2131,
 4,
 65,
 4,
 37,
 5020,
 4,
 34,
 4,
 56,
 51,
 65,
 35,
 35,
 35,
 35,
 35,
 65,
 2131,
 65,
 4,
 35,
 1800,
 65,
 1800,
 4,
 4,
 4,
 4,
 35,
 6011,
 305,
 35,
 4,
 35,
 37,
 300,
 4,
 4,
 34,
 4,
 4,
 35,
 300,
 51,
 35,
 4,
 1800,
 4,
 35,
 4,
 4,
 4,
 4,
 300,
 1800,
 6011,
 35,
 35,
 4,
 4,
 35,
 35,
 304,
 6011,
 4,
 55,
 6011,
 1800,
 4,
 4,
 51,
 35,
 35,
 304,
 4,
 37,
 38,
 6011,
 35,
 4,
 4,
 4,
 37,
 4,
 35,
 4,
 35,
 35,
 2131,
 6011,
 35,
 65,
 4,
 34,
 35,
 35,
 4,
 35,
 65,
 51,
 4,
 2131,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 300,
 6011,
 1800,
 4,
 1800,
 4,
 38,
 4,
 56,
 65,
 4,
 4,
 35,
 35,
 4,
 4,
 4,
 4,
 35,
 35,
 35,
 6011,
 35,
 1800,
 1800,
 52,
 4,
 35,
 4,
 4,
 1800,
 36,
 65,
 4,
 35,
 5038,
 4,
 35,
 4,
 4,
 34,
 6011,
 35,
 1800,
 300,
 4,
 4,
 4,
 37,
 4,
 4,
 305,
 4,
 56,
 4,
 35,
 52,


In [158]:
def filter_transactions(transaction_files, credit_cards, start_vals):
    """
    Takes in a pandas dataframe and a list of credit card numbers 
    and filters the dataframe to create a new column 
    based on if the credit_card_number starts with one of the 
    valid prefix vendor values 
    """
    numbers = transaction_files["credit_card_number"]
    str_nums = []
    valid_nums = []
    for index, number in enumerate(numbers):
        str_num = str(number)
        result = list(filter(str_num.startswith, credit_cards)) != [] # if the element startswith a credit_cardscredit_card_numbers value keep it, otherwise, remove 
        valid_nums.append(result) # this a boolean list of 'True' or 'False' if the card matches the valid vendor numbers 
        str_nums.append(str_num) # This is a list of all the transaction values as strings 
    results = pd.DataFrame({'credit_card_number':str_nums,
                            'ipv4':transaction_files['ipv4'],
                            'state':transaction_files['state'],
                            'valid_vendor':valid_nums})
    valid_results = results[results['valid_vendor'] == True]
    valid_results = valid_results.assign(start_val = start_vals)
    valid_results = valid_results.reset_index(drop=True)
    
    fraud_results = results[results['valid_vendor'] == False]
    fraud_results = fraud_results.reset_index(drop=True)
    return results, numbers, valid_results, fraud_results


results, numbers, valid_results, fraud_results = filter_transactions(transaction_files, credit_cards, start_vals)


In [181]:
fraud_results

,credit_card_number,ipv4,state,valid_vendor
0,30213196611688,192.168.216.212,AZ,False
1,5393825843017645,192.168.211.229,OK,False
2,580022764071,172.27.166.188,FL,False
3,30334856785990,10.15.215.220,SD,False
4,5383983915126000,192.168.30.242,ID,False
...,...,...,...,...
46370,630449879927,172.21.198.100,OR,False
46371,60417923400,10.234.170.145,MI,False
46372,676344351470,10.141.151.33,KY,False
46373,676270429225,10.34.224.82,NY,False


In [180]:
valid_results

,credit_card_number,ipv4,state,valid_vendor,start_val
0,4938684086769,172.29.189.27,CA,True,4
1,213128820373753,172.22.174.184,AR,True,2131
2,3554681727155351,172.24.27.121,ID,True,35
3,4339158798023,10.87.17.223,SC,True,4
4,341265497366150,10.134.18.19,AR,True,34
...,...,...,...,...,...
453620,4407827161521023,192.168.92.89,NM,True,4
453621,4797462133093618,10.228.168.20,WI,True,4
453622,3595014020561169,172.21.190.37,RI,True,35
453623,30416095371458,192.168.68.115,MO,True,304


## Part II 

- Candidate should assume that going forward, only the sanitized dataset should be used.


### - Find in the sanitized dataset if it contains fraudulent transactions (using the data from loading fraud.zip) and report their number.


In [185]:
def filter_fraud_in_transactions(fraud_dataset, valid_results):
    """
    From the santitized results dataframe 
    filter the DF to display only the numbers
    that are present in the fraud.zip file 
    """
#     This just produces a list of all the fraud credit_cards that exist in the sanitized dataframe
#     fraud_transactions = list(set(results_filtered['credit_card_number']).intersection(fraud_dataset))

#     A better way is to produce the whole dataframe and other columns of the credit_cardscredit_card_numbers in the sanitized
#     dataframe that also exist in fraud.zip 
    fraud_transactions = valid_results[valid_results['credit_card_number'].isin(fraud_dataset)].drop_duplicates()
    fraud_transactions = fraud_transactions.reset_index(drop=True)
    fraud_transactions['valid_card'] = False
    valid_transactions = valid_results.drop(fraud_transactions.index)
    valid_transactions['valid_card'] = True
    return valid_transactions, fraud_transactions

In [186]:
valid_transactions, fraud_transactions = filter_fraud_in_transactions(fraud_dataset, valid_results)
fraud_transactions

,credit_card_number,ipv4,state,valid_vendor,start_val,valid_card
0,4013901026491,172.23.51.228,NV,True,4,False
1,5128258542512744,10.65.191.187,NJ,True,51,False
2,4673436419900968,10.68.149.54,ND,True,4,False
3,6011531195808785,192.168.216.58,MA,True,6011,False
4,3518829214836604,10.55.8.196,AK,True,35,False
...,...,...,...,...,...,...
922,30443556780787,10.155.131.204,IL,True,304,False
923,375712024591957,192.168.99.32,MS,True,37,False
924,3521881287540980,192.168.248.225,LA,True,35,False
925,3574961180083290,172.17.123.72,ND,True,35,False


In [182]:
valid_transactions

,credit_card_number,ipv4,state,valid_vendor,start_val,valid_card
927,341699761913307,172.25.148.118,IN,True,34,True
928,180011131174905,192.168.38.179,PA,True,1800,True
929,4808375377167,172.16.48.188,GA,True,4,True
930,4864308841346749,10.145.199.199,LA,True,4,True
931,3541314305340585,192.168.117.215,TX,True,35,True
...,...,...,...,...,...,...
453620,4407827161521023,192.168.92.89,NM,True,4,True
453621,4797462133093618,10.228.168.20,WI,True,4,True
453622,3595014020561169,172.21.190.37,RI,True,35,True
453623,30416095371458,192.168.68.115,MO,True,304,True


In [53]:
model_data = valid_transactions.append(fraud_transactions)
# model_data.to_csv('modeling_data.csv')

In [163]:
print('Within the sanitized dataframe, there were', len(fraud_transactions), 'instances of fraudulent transactions.')

Within the sanitized dataframe, there were 927 instances of fraudulent transactions.


### - Create a report of the number of fraudulent transactions per state.

In [164]:
# - Create a report of the number of fraudulent transactions per state.
### MAY WANT TO CONSIDER RETURNING A JSON FORMAT FILE 

def get_fraud_by_state(fraud_transactions):
    """
    Takes in a dataframe (fraud_transactions within valid_results)
    performs a group by on state,
    and returns a json serialized list of the count of
    transactions by state as objects
    """
    fraud_groupby_state = fraud_transactions.groupby('state').count()
    fraud_by_state = fraud_groupby_state[['credit_card_number']]
    fraud_by_state = fraud_by_state.rename(columns={'credit_card_number': 'count'})

    return fraud_by_state
fraud_by_state = get_fraud_by_state(fraud_transactions)

In [56]:
# print(fraud_by_state.sum())
# fraud_by_state[0:10]

### - Create a report of the number of fraudulent transactions per card vendor, e.g., maestro => 45, amex => 78, etc..


In [57]:
def get_vendors_val_key(item):
    for key, val in vendors.items():
        if item in val:
            return key

In [58]:
# - Create a report of the number of fraudulent transactions per vendor.
### MAY WANT TO CONSIDER RETURNING A JSON FORMAT FILE 

def get_fraud_by_vendor(fraud_transactions):
    """
    Takes in an dataframe performs a group by on state,
    and returns a json serialized list of the count of
    transactions by state as objects
    """
    fraud_transactions['start_val']= fraud_transactions['start_val'].map(str)
    fraud_transactions['name'] = fraud_transactions.start_val.map(get_vendors_val_key)
    fraud_groupby_vendor = fraud_transactions.groupby('name').count()
    fraud_by_vendor = fraud_groupby_vendor[['credit_card_number']]
    fraud_by_vendor = fraud_by_vendor.rename(columns={'credit_card_number': 'count'})
    return fraud_transactions, fraud_by_vendor

In [187]:
fraud_transactions

,credit_card_number,ipv4,state,valid_vendor,start_val,valid_card
0,4013901026491,172.23.51.228,NV,True,4,False
1,5128258542512744,10.65.191.187,NJ,True,51,False
2,4673436419900968,10.68.149.54,ND,True,4,False
3,6011531195808785,192.168.216.58,MA,True,6011,False
4,3518829214836604,10.55.8.196,AK,True,35,False
...,...,...,...,...,...,...
922,30443556780787,10.155.131.204,IL,True,304,False
923,375712024591957,192.168.99.32,MS,True,37,False
924,3521881287540980,192.168.248.225,LA,True,35,False
925,3574961180083290,172.17.123.72,ND,True,35,False


In [60]:
fraud_transactions, fraud_by_vendor = get_fraud_by_vendor(fraud_transactions)
print(fraud_by_vendor.sum())
# fraud_by_vendor

count    927
dtype: int64


### - Create a dataset of 3 columns and save in both JSON and in a binary fileformat that you believe it's suitable for BI analysis:
####   - column 1: masked credit card: replace 9 last digits of the credit card with `*`
####   - column 2: ip address
####   - column 3: state
####   - column 4: sum of number of byte of (column 1 + column 2 + column 3)

In [61]:
def masked_credit_cards(valid_results):
    dicts = []
    for index, row in enumerate(valid_results['credit_card_number']):
        full_card_number = valid_results['credit_card_number'].astype(str).iloc[index][:-9]
        masked = full_card_number + '*' * 9
        ipv4_value = valid_results['ipv4'][index]
        state_value = valid_results['state'][index]
        byte_number = sys.getsizeof(masked + ipv4_value + state_value)
        dict = {'credit_card_number' : masked,
                 'ipv4' : ipv4_value,
                 'state' : state_value,
                 'bytes' : byte_number}
        dicts.append(dict)
    df_dicts = pd.DataFrame(dicts) 
    return df_dicts

In [ ]:
# tmp = fraud_transactions[0:10]

In [63]:
# %%time
# final_df = masked_credit_cards(valid_results)
# final_df

CPU times: user 52min 23s, sys: 3.8 s, total: 52min 27s
Wall time: 1h 10min 29s


,credit_card_number,ipv4,state,bytes
0,4938*********,172.29.189.27,CA,77
1,213128*********,172.22.174.184,AR,80
2,3554681*********,172.24.27.121,ID,80
3,4339*********,10.87.17.223,SC,76
4,341265*********,10.134.18.19,AR,78
...,...,...,...,...
453620,4407827*********,192.168.92.89,NM,80
453621,4797462*********,10.228.168.20,WI,80
453622,3595014*********,172.21.190.37,RI,80
453623,30416*********,192.168.68.115,MO,79


In [66]:
final_df = masked_credit_cards(valid_results)
final_df

,credit_card_number,ipv4,state,bytes
0,4938*********,172.29.189.27,CA,77
1,213128*********,172.22.174.184,AR,80
2,3554681*********,172.24.27.121,ID,80
3,4339*********,10.87.17.223,SC,76
4,341265*********,10.134.18.19,AR,78
...,...,...,...,...
453620,4407827*********,192.168.92.89,NM,80
453621,4797462*********,10.228.168.20,WI,80
453622,3595014*********,172.21.190.37,RI,80
453623,30416*********,192.168.68.115,MO,79


In [64]:
def make_json_file(final_df):
    directory = 'results/'
    sample = final_df.to_dict()
    with open(directory + "final_df.json", "w") as outfile:
        json.dump(sample, outfile)

In [65]:
make_json_file(final_df)

In [188]:
def make_binary_file(final_df):
    directory = 'results/'
    with open(directory + 'sample', 'wb') as outfile:
        outfile.write(bytes(final_df, encoding='utf8'))
    

In [189]:
make_binary_file(final_df)

TypeError: encoding without a string argument

In [ ]:
def generate_json_file(data, file):
    directory = 'results/'
    with open(directory + file + '.json', 'w') as f:
        f.write(data)

def generate_binary_file(data, file):
    directory = 'results/'
    with open(directory + file, 'wb') as f:
        f.write(bytes(data, encoding='utf8'))

### Part IV 

#### - Create a ML model that you think is useful using the datasets provided (raw or/and sanitized)
#### - Provide insight/metrics on how to validate if the model is accurate.

In [ ]:
# frauddata = ('/Users/sarahkaunitz/Documents/deloitte/AI_Engineer/code_challenge/datasets/modeling_data.csv')
# fraud_data = pd.read_csv(frauddata)
# fraud_data.drop(['Unnamed: 0'], axis=1)